In [1]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint
import data

In [43]:
vocab_size = 324
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
x, y = data.sample(d, 5)
print(x.shape, y.shape)

324
((5, 324), (5, 324))


In [53]:
# dirrs = ['sentiment/train/', 'sentiment/test/']
# sent = []
# for dirr in dirrs:
#     print dirr
#     l = listdir(dirr+'pos')
#     print 'pos'
#     for r in l:
#         t = codecs.open(dirr+'pos/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))
#     l = listdir(dirr+'neg')
#     print 'neg'
#     for r in l:
#         t = codecs.open(dirr+'neg/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))

In [54]:
# m = Word2Vec.load('embedding.model')
# data = []
# for s in sent:
#     a = [m[x] for x in s]
#     data.append(a)
# data = np.array(data)

In [55]:
# def sample(d):
#     r = randint(0, data.shape[0] - 1)
#     t = d[r]
#     x = t[:-1]
#     y = t[1:]
#     return np.array(x), np.array(y)

# s = sample(data)
# s[0].shape, s[1].shape

In [33]:
# hyper-parameters
epochs = 80000
hidden_layer = 100
inp_out_size = 324

In [28]:
# Wxh = tf.Variable([[.1, .2], [.3, .4], [.5, .6], [.7, .8]], dtype=tf.float32)
# Whh = tf.Variable([[.1, .2], [.3, .4]], dtype=tf.float32)
# Why = tf.Variable([[.1, .2, .3, .4], [.4, .5, .6, .7]], dtype=tf.float32)
Wxh = tf.Variable(tf.random_uniform([inp_out_size, hidden_layer], minval=-0.1, maxval=0.1))
Whh = tf.Variable(tf.random_uniform([hidden_layer, hidden_layer], minval=-0.1, maxval=0.1))
Why = tf.Variable(tf.random_uniform([hidden_layer, inp_out_size], minval=-0.1, maxval=0.1))

In [29]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    h = tf.nn.tanh(tf.squeeze(tf.matmul(i, Wxh)) + tf.squeeze(tf.matmul(p, Whh)))
    return h

In [47]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
# initial = tf.Variable([1, 2], dtype=tf.float32)
initializer = tf.random_uniform_initializer(-0.1, 0.1)
with tf.variable_scope('initial', initializer=initializer):
    initial = tf.Variable(tf.zeros([hidden_layer]), dtype=tf.float32)
states = tf.scan(recurrence, a, initializer=initial)
outputs = tf.nn.softmax(tf.matmul(states, Why))
loss = tf.reduce_sum(- b * tf.log(outputs))
# loss = tf.sqrt(tf.reduce_mean(tf.square(tf.sub(outputs, b))))
trainables = tf.trainable_variables()
grads = tf.gradients(loss, trainables)
grads, _ = tf.clip_by_global_norm(grads, clip_norm=1.0)
grad_var_pairs = zip(grads, trainables)
optimizer = tf.train.AdagradOptimizer(0.01)
optimize_op = optimizer.minimize(loss)

In [ ]:
print epochs
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(epochs):
        x, y = data.sample(d, 10)
        l, s, _ = sess.run([loss, states, optimize_op], {a: x, b: y})
        if i % 3000 == 0:
            print 'epoch {0}, loss = {1}'.format(i, l)
            print s[:,0]
    
    x, y = data.sample(d, 50)
    o, s = sess.run([outputs, states], {a: x, b: y})
    print s

80000
epoch 0, loss = 57.8063049316
[-0.0750367  -0.11987324  0.07136356  0.00827659 -0.06204062  0.05491559
  0.06906988 -0.10053468  0.02205851 -0.12003911]


In [ ]:
np.array(data[0]).shape